<a href="https://colab.research.google.com/github/ssgalitsky/PedalNetRT/blob/master/colab_TPU_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



links:

PyTorch Lightning Documentation https://pytorch-lightning.readthedocs.io/en/stable/

In [33]:
#@title 1. clone  https://github.com/ssgalitsky/PedalNetRT.git
!rm -rf '/content/PedalNetRT'
import os
os.chdir('/content')
!git clone https://github.com/ssgalitsky/PedalNetRT.git


Cloning into 'PedalNetRT'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 224 (delta 109), reused 27 (delta 3), pack-reused 0
Receiving objects: 100% (224/224), 158.40 MiB | 15.98 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [ ]:
#@title 2.1 install dependencies
!pip3 install torchvision
!pip3 install pytorch_lightning==0.9.0
use_tpu=False

In [ ]:
#@title 2.2 install optional dependencies (TPU)
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl
use_tpu=True

In [3]:
import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()
use_tpu=True
#from google.colab import drive
#drive.mount('/content/drive')




In [40]:
#@title 3. prepare data
os.chdir('/content/PedalNetRT')
!python3 "prepare_data.py" "/content/PedalNetRT/data/s chain1.wav" "/content/PedalNetRT/data/s chain5.wav"


In [ ]:
#@title 4. **train model** Beware, checkpoint files not visible via Colab webui (-rw-r--r--) 
import os
if use_tpu:
  assert os.environ['COLAB_TPU_ADDR']
os.chdir('/content/PedalNetRT')  
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 --kernel_size=2
#!python3 "train.py" --batch_size=32 --max_epochs=100 --learning_rate=3e-3 --num_channels=5 






2020-10-15 00:00:02.197580: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
GPU available: False, used: False
TPU available: True, using: 8 TPU cores
training on 8 TPU cores
Traceback (most recent call last):
  File "train.py", line 43, in <module>
    main(args)
  File "train.py", line 22, in main
    trainer.fit(model)
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/states.py", line 48, in wrapped_fn
    result = fn(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/trainer.py", line 1078, in fit
    self.accelerator_backend.train(model)
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/accelerators/tpu_backend.py", line 87, in train
    start_method=self.start_method
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 395, in spawn
    start_method=start_method)
  File "/usr/local/lib

In [42]:
!ls -d /content/PedalNetRT/lightning_logs/version_1/checkpoints/*

#!chmod -R 755 lightning_logs 


'/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=782.ckpt'


In [43]:
#@title 5. convert torch model (ckpt) to plugin model (json) 
# The .ckpt model must be converted to a .json model to run in the plugin. Usage:

#!python3 convert_pedalnet_to_wavnetva.py --model=your_trained_model.ckpt
!python3 convert_pedalnet_to_wavenetva.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=782.ckpt





2020-10-14 23:53:45.391525: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
^C


In [44]:
!python test.py --model=/content/PedalNetRT/lightning_logs/version_1/checkpoints/epoch=782.ckpt

2020-10-14 23:56:15.062392: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "test.py", line 7, in <module>
    from model import PedalNet
  File "/content/PedalNetRT/model.py", line 6, in <module>
    import pytorch_lightning as pl
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/__init__.py", line 59, in <module>
    from pytorch_lightning.trainer import Trainer
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/__init__.py", line 1093, in <module>
    from pytorch_lightning.trainer.trainer import Trainer
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/trainer.py", line 32, in <module>
    from pytorch_lightning.loggers import LightningLoggerBase
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/loggers/__init__.py", line 5, in <module>
    from pytorch_lightning.loggers.tensorboard import TensorBo

In [39]:
!python plot_wav.py

Error to signal (with pre-emphasis filter):  0.80168515
Error to signal (no pre-emphasis filter):  0.80168515
Creating spectrogram data..
plot_wav.py:116: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 10*np.log10(spectrogram))
<Figure size 1300x800 with 3 Axes>
<Figure size 1200x800 with 2 Axes>


In [ ]:
#@title 6. convert model from TPU to CPU format
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
